# サーチエンジンとしての利用手順(データ蓄積)
----
LogstashからElasticsearchへ、様々な形式の データを蓄積する方法について説明します。

## 準備

本章のコマンドを実行するための設定を行います。  

セットアップ済みの環境のうちコマンドを送る先であるCoordinating(Client) Nodeのホストアドレスを、次のセルに記入して実行し、保存してください。

In [1]:
%run rc.py
%env ES_HOST={es_host}:9200

env: ES_HOST=10.24.128.93:9200


また、データを蓄積する先のindexとtypeを、次のセルを実行して保存してください。  

※特にセルの内容の書き換えは必要ありません。  
　前の章と異なるindex名にしているのは、この章のコマンドで投入する内容が前の章までのものと混ざらないようにするためです。

In [2]:
%env INDEX=sample_index
%env TYPE=logs

env: INDEX=sample_index
env: TYPE=logs


## Logstashとは

Logstashは、Elastic Stackを構成するプロダクトの１つです。  
データの取り込み、加工、出力（格納）を処理するパイプラインを簡単に構築できます。

対応できる形式は次のように多岐に渡ります。

- 入力：各種ファイル、RDB、通信ストリーム、ログ、S3、BeatsというElasticのデータ収集ツール…
- 加工：各種形式の解釈、分割、集計、地理情報の付与、個人情報のマスク…
- 出力：Elasticsearch、S3、各種ファイル、通信…

対応できる形式などについては、Logstash([公式ページ](https://www.elastic.co/jp/products/logstash))を参照してください。  
  
本章では、入力に各種ファイル、格納先にElasticsearchを指定する、単純なケースでのデータ蓄積を実施します。  
また、変換処理をLogstashで実施するだけでなく、ElasticsearchのIngest Nodeで分散実施するケースについても説明します。


## Logstashの設定

Logstashを利用する際には、".conf"という拡張子のファイルに振る舞いを定義し、それを読み込んで実行させます。  
sample.confというconfファイルを用いる場合の実行コマンドは次のようになります。

confファイルはinput・filter・outputの３つの設定によって構成されます。  
- **input** : 入力元や形式の設定
- **filter** : 入力された内容の加工内容の設定
- **output** : 出力先や形式の設定

confファイルの例を次に示します。

```ruby:example.conf
input{
  stdin{
    codec => plain{
      charset => "UTF-8"
    }
  }
}
filter{
  grok { 
    match => [ "message", "%{COMBINEDAPACHELOG}" ] 
  }
  geoip{
    source => "ip"
  }
}
output{
  stdout{
    codec => dots
  }
  elasticsearch{
    hosts => localhost
    user => "elastic"
    password => "changeme"
    index => "apache_sample"
  }
}
```

設定ファイルの各要素の内容や、上の例で対応する部分の意味は次の通りです。

||意味|設定例の内容|設定内容の参考ページ|
|----|----|----|----|
|**input**|データの入力元や形式、およびその詳細を定義します。|stdin{}は標準入力からデータを受けとる設定です。<br>また、文字コードをUTF-8に指定しています。<br>stdin{}の代わりにtwitter{},file{}のように指定すれば入力元を標準入力以外にすることもできます。|[Input plugins](https://www.elastic.co/guide/en/logstash/current/input-plugins.html)|
|**filter**|inputから受け取った入力の加工内容を定義します。<br>複数のfilterを記述した場合、記述順に加工されます。|入力の特定部分をパースするgrok filterと<br>地理情報を付加するgeoip filterを指定しています。|[Filter plugins](https://www.elastic.co/guide/en/logstash/current/filter-plugins.html)|
|**output**|データの出力先や形式、およびその詳細を定義します。|stdout{}とelasticsearch{}の2つを記述することで、標準出力とElasticsearchの両方に出力するよう指定しています。<br>Elasticsearchへの出力設定では、宛先ホストやアカウント、index名を指定しています。|[Output plugins](https://www.elastic.co/guide/en/logstash/current/output-plugins.html)|

## データフォーマット

ここでは、様々なデータフォーマットのデータをElasticsearchに登録する際の具体的な設定と手法について説明します。

### JSON

JSON形式のデータとして、次の内容のサンプルを登録します。

In [3]:
!head -n 1 sample_data/weather.json

{"timestamp":"1","atmospheric_pressure":"1000.4","sea_level_pressure":"1003.4","precipitation_day":"0","precipitation_max_hour":"0","precipitation_max_10min":"0","temperature_avg":"3.7","temperature_max":"8.2","temperature_min":"0.7","humidity_avg":"41","humidity_min":"30","wind_speed_avg":"4.4","wind_speed_max":"9.4","wind_direction":"西","wind_speed_max_moment":"18.7","wind_direction_max_moment":"北西","sunshine_duration":"1.3","snowfall":"","snowfall_max":"","information_daytime":"時々晴一時雪","information_night":"晴"}


confファイルには、codec => "json" と指定することでJSON形式を解釈できるように指定します。  
ファイル名は"json.conf"とします。

```ruby:json.conf
input{  
  stdin{ 
    codec => "json"  
  }  
}  
filter{  
}  
output{  
  elasticsearch{  
    hosts => <ES_HOST>  
    index => <INDEX>  
  }  
}  
```


weather.jsonとjson.confをLogstashのサーバーに配置してください。

In [4]:
!ansible logstash-server -i ./hosts -m file -a "path=~/sample_data state=directory" -u $USER --private-key=$KEYPATH
!ansible logstash-server -i ./hosts -m file -a "path=~/sample_conf state=directory" -u $USER --private-key=$KEYPATH
!ansible logstash-server -i ./hosts -m copy -a "src=sample_data/weather.json dest=~/sample_data/weather.json" -u $USER --private-key=$KEYPATH
!ansible logstash-server -i ./hosts -m copy -a "src=logstash_conf/json.conf dest=~/sample_conf/json.conf" -u $USER --private-key=$KEYPATH

10.24.128.93 | SUCCESS => {
    "changed": true, 
    "gid": 1000, 
    "group": "centos", 
    "mode": "0775", 
    "owner": "centos", 
    "path": "/home/centos/sample_data", 
    "secontext": "unconfined_u:object_r:user_home_t:s0", 
    "size": 6, 
    "state": "directory", 
    "uid": 1000
}
10.24.128.93 | SUCCESS => {
    "changed": true, 
    "gid": 1000, 
    "group": "centos", 
    "mode": "0775", 
    "owner": "centos", 
    "path": "/home/centos/sample_conf", 
    "secontext": "unconfined_u:object_r:user_home_t:s0", 
    "size": 6, 
    "state": "directory", 
    "uid": 1000
}
10.24.128.93 | SUCCESS => {
    "changed": true, 
    "checksum": "b7e7b6ede3b70b89dcd9a332e37f4136b6ca91ca", 
    "dest": "/home/centos/sample_data/weather.json", 
    "gid": 1000, 
    "group": "centos", 
    "md5sum": "da7cb75de960c8251f24fd036b371941", 
    "mode": "0664", 
    "owner": "centos", 
    "secontext": "unconfined_u:object_r:user_home_t:s0", 
    "size": 16447, 
    "src": "/home/centos/

実際のデータ登録は次のコマンドで実行します。

In [5]:
!ansible logstash-server -i ./hosts -m shell -a "cat ~/sample_data/weather.json | sudo /usr/share/logstash/bin/logstash --path.settings /etc/logstash -f ~/sample_conf/json.conf" -u $USER --private-key=$KEYPATH

10.24.128.93 | SUCCESS | rc=0 >>
Sending Logstash logs to /var/log/logstash which is now configured via log4j2.properties.



コマンドの実行に成功したら、次の検索APIで正しく登録されているか確認してください。  
正しく登録されていれば、json形式で登録されたデータが出力されます。

In [6]:
%%bash
curl -XGET "http://$ES_HOST/$INDEX/sample_json/_search?pretty"

{
  "took" : 2,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "failed" : 0
  },
  "hits" : {
    "total" : 30,
    "max_score" : 1.0,
    "hits" : [
      {
        "_index" : "sample_index",
        "_type" : "sample_json",
        "_id" : "AVyGu-TXdDy3gmLO23ZT",
        "_score" : 1.0,
        "_source" : {
          "precipitation_max_10min" : "0",
          "wind_speed_avg" : "4.4",
          "snowfall" : "",
          "precipitation_max_hour" : "0",
          "temperature_max" : "8.2",
          "wind_direction" : "西",
          "information_daytime" : "時々晴一時雪",
          "precipitation_day" : "0",
          "@timestamp" : "2017-06-08T08:04:39.318Z",
          "humidity_avg" : "41",
          "humidity_min" : "30",
          "wind_speed_max" : "9.4",
          "sunshine_duration" : "1.3",
          "temperature_avg" : "3.7",
          "temperature_min" : "0.7",
          "@version" : "1",
          "host" : "0.0.0.0",
          "atmospheric_pres

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11085  100 11085    0     0  1745k      0 --:--:-- --:--:-- --:--:-- 1804k


### 複数行JSON

1つのJSONデータが改行を含んだ形式の場合を試します。  
次の内容のサンプルを登録します。

In [7]:
!head -n 3 sample_data/weather_multiline.json

{
"timestamp":"1",
"atmospheric_pressure":"1000.4",


confファイルには、codec => multilineと指定することで、複数行にまたがるデータ形式を解釈できるように指定します。
通常、Logstashでは1行ごとに1つのイベントとして処理しますが、multilineでは特定のパターンを見つけるまで行を読み込み、結合することで１つのイベントとして処理します。  

ファイル名は"json_multiline.conf"とします。

```rubby:weather_multiline.conf
input{
  stdin{
    codec => multiline{
      pattern => "}"
      negate => "true"
      what => "next"
    }
  }
}
filter{
  json{
    source => "message"
  }
}
output{
  elasticsearch{
    hosts => <ES_HOST>
    index => <INDEX>
  }
}
```

実際のデータ登録は次のコマンドで実行します。

In [8]:
!ansible logstash-server -i ./hosts -m file -a "path=~/sample_data state=directory" -u $USER --private-key=$KEYPATH
!ansible logstash-server -i ./hosts -m file -a "path=~/sample_conf state=directory" -u $USER --private-key=$KEYPATH
!ansible logstash-server -i ./hosts -m copy -a "src=sample_data/weather_multiline.json dest=~/sample_data/weather_multiline.json" -u $USER --private-key=$KEYPATH
!ansible logstash-server -i ./hosts -m copy -a "src=logstash_conf/json_multiline.conf dest=~/sample_conf/json_multiline.conf" -u $USER --private-key=$KEYPATH

10.24.128.93 | SUCCESS => {
    "changed": false, 
    "gid": 1000, 
    "group": "centos", 
    "mode": "0775", 
    "owner": "centos", 
    "path": "/home/centos/sample_data", 
    "secontext": "unconfined_u:object_r:user_home_t:s0", 
    "size": 25, 
    "state": "directory", 
    "uid": 1000
}
10.24.128.93 | SUCCESS => {
    "changed": false, 
    "gid": 1000, 
    "group": "centos", 
    "mode": "0775", 
    "owner": "centos", 
    "path": "/home/centos/sample_conf", 
    "secontext": "unconfined_u:object_r:user_home_t:s0", 
    "size": 22, 
    "state": "directory", 
    "uid": 1000
}
10.24.128.93 | SUCCESS => {
    "changed": true, 
    "checksum": "595d7c33b1630229141fec243e7622008b4bf49a", 
    "dest": "/home/centos/sample_data/weather_multiline.json", 
    "gid": 1000, 
    "group": "centos", 
    "md5sum": "0beb995c33578c7266262f617c65d87a", 
    "mode": "0664", 
    "owner": "centos", 
    "secontext": "unconfined_u:object_r:user_home_t:s0", 
    "size": 17129, 
    "src": 

In [9]:
!ansible logstash-server -i ./hosts -m shell -a "cat ~/sample_data/weather_multiline.json | sudo /usr/share/logstash/bin/logstash --path.settings /etc/logstash -f ~/sample_conf/json_multiline.conf" -u $USER --private-key=$KEYPATH

10.24.128.93 | SUCCESS | rc=0 >>
Sending Logstash logs to /var/log/logstash which is now configured via log4j2.properties.



コマンドの実行に成功したら、次の検索APIで正しく登録されているか確認してください。  
正しく登録されていれば、json形式で登録されたデータが出力されます。

In [10]:
%%bash
curl -XGET "http://$ES_HOST/$INDEX/sample_multiline_json/_search?pretty"

{
  "took" : 2,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "failed" : 0
  },
  "hits" : {
    "total" : 31,
    "max_score" : 1.0,
    "hits" : [
      {
        "_index" : "sample_index",
        "_type" : "sample_multiline_json",
        "_id" : "AVyGvC4bdDy3gmLO23Zx",
        "_score" : 1.0,
        "_source" : {
          "wind_speed_avg" : "4.4",
          "snowfall" : "",
          "temperature_max" : "8.2",
          "precipitation_day" : "0",
          "wind_speed_max" : "9.4",
          "sunshine_duration" : "1.3",
          "temperature_avg" : "3.7",
          "temperature_min" : "0.7",
          "@version" : "1",
          "host" : "0.0.0.0",
          "wind_direction_max_moment" : "北西",
          "information_night" : "晴",
          "timestamp" : "1",
          "precipitation_max_10min" : "0",
          "precipitation_max_hour" : "0",
          "wind_direction" : "西",
          "message" : "{\n\"timestamp\":\"1\",\n\"atmospheric_pressu

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 18564  100 18564    0     0  2663k      0 --:--:-- --:--:-- --:--:-- 3021k


### CSV


CSV形式のデータとして、次の内容のサンプルを登録します。

In [11]:
!head -n 3 sample_data/weather.csv

1,1000.4,1003.4,0,0,0,3.7,8.2,0.7,41,30,4.4,9.4,西,18.7,北西,1.3,,,時々晴一時雪,晴
2,1007.4,1010.4,,,,2.7,7.9,-2.2,41,19,2.1,4.2,西,7.8,西,6,,,時々曇,一時曇
3,1012,1015.1,,,,3.8,8.9,-1.1,42,19,3,7.4,北西,10.9,北西,8.9,,,晴,後薄曇


confファイルには、"filter"の下に"csv"定義を設定し、列ごとの名前を指定することで、CSVデータの各列を対応する要素に紐づけてCSV形式を解釈できるようになります。  
ファイル名は"csv.conf"とします。

```ruby:csv.conf
input{
   stdin{}
}
filter{
  csv{
    columns => ["timestamp","atmospheric_pressure","sea_level_pressure","precipitation_day","precipitation_max_hour","precipitation_max_10min","temperature_avg","temperature_max","temperature_min","humidity_avg","humidity_min","wind_speed_avg","wind_speed_max","wind_direction","wind_speed_max_moment","wind_direction_max_moment","sunshine_duration","snowfall","snowfall_max","information_daytime","information_night"]
  }
}
output{
  elasticsearch{
    hosts => <ES_HOST>  
    index => <INDEX>
  }
}
```

In [12]:
!ansible logstash-server -i ./hosts -m file -a "path=~/sample_data state=directory" -u $USER --private-key=$KEYPATH
!ansible logstash-server -i ./hosts -m file -a "path=~/sample_conf state=directory" -u $USER --private-key=$KEYPATH
!ansible logstash-server -i ./hosts -m copy -a "src=sample_data/weather.csv dest=~/sample_data/weather.csv" -u $USER --private-key=$KEYPATH
!ansible logstash-server -i ./hosts -m copy -a "src=logstash_conf/csv.conf dest=~/sample_conf/csv.conf" -u $USER --private-key=$KEYPATH

10.24.128.93 | SUCCESS => {
    "changed": false, 
    "gid": 1000, 
    "group": "centos", 
    "mode": "0775", 
    "owner": "centos", 
    "path": "/home/centos/sample_data", 
    "secontext": "unconfined_u:object_r:user_home_t:s0", 
    "size": 54, 
    "state": "directory", 
    "uid": 1000
}
10.24.128.93 | SUCCESS => {
    "changed": false, 
    "gid": 1000, 
    "group": "centos", 
    "mode": "0775", 
    "owner": "centos", 
    "path": "/home/centos/sample_conf", 
    "secontext": "unconfined_u:object_r:user_home_t:s0", 
    "size": 48, 
    "state": "directory", 
    "uid": 1000
}
10.24.128.93 | SUCCESS => {
    "changed": true, 
    "checksum": "c93efa572d2e1a05cf453c051587d8893247b5b7", 
    "dest": "/home/centos/sample_data/weather.csv", 
    "gid": 1000, 
    "group": "centos", 
    "md5sum": "dab98693b329f1917542f85d475c36af", 
    "mode": "0664", 
    "owner": "centos", 
    "secontext": "unconfined_u:object_r:user_home_t:s0", 
    "size": 2621, 
    "src": "/home/cento

実際のデータ登録は次のコマンドで実行します。

In [13]:
!ansible logstash-server -i ./hosts -m shell -a "cat ~/sample_data/weather.csv | sudo /usr/share/logstash/bin/logstash --path.settings /etc/logstash -f ~/sample_conf/csv.conf" -u $USER --private-key=$KEYPATH

10.24.128.93 | SUCCESS | rc=0 >>
Sending Logstash logs to /var/log/logstash which is now configured via log4j2.properties.



コマンドの実行に成功したら、次の検索APIで正しく登録されているか確認してください。  
正しく登録されていれば、json形式で登録されたデータが出力されます。

In [14]:
%%bash
curl -XGET "http://$ES_HOST/$INDEX/sample_csv/_search?pretty"

{
  "took" : 2,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "failed" : 0
  },
  "hits" : {
    "total" : 30,
    "max_score" : 1.0,
    "hits" : [
      {
        "_index" : "sample_index",
        "_type" : "sample_csv",
        "_id" : "AVyGvHe2dDy3gmLO23aQ",
        "_score" : 1.0,
        "_source" : {
          "wind_speed_avg" : "4.4",
          "snowfall" : null,
          "temperature_max" : "8.2",
          "precipitation_day" : "0",
          "wind_speed_max" : "9.4",
          "sunshine_duration" : "1.3",
          "temperature_avg" : "3.7",
          "temperature_min" : "0.7",
          "@version" : "1",
          "host" : "0.0.0.0",
          "wind_direction_max_moment" : "北西",
          "information_night" : "晴",
          "timestamp" : "1",
          "precipitation_max_10min" : "0",
          "precipitation_max_hour" : "0",
          "wind_direction" : "西",
          "message" : "1,1000.4,1003.4,0,0,0,3.7,8.2,0.7,41,30,4.4,9.4,西,18.7

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 12224  100 12224    0     0  2356k      0 --:--:-- --:--:-- --:--:-- 2984k


### LTSV

LTSV形式のデータとして、次の内容のサンプルを登録します。

In [15]:
!head -n 3 sample_data/weather.ltsv

"timestamp":"21"	"atmospheric_pressure":"1024.3"	"sea_level_pressure":"1027.4"	"precipitation_day":"4"	"precipitation_max_hour":"1.5"	"precipitation_max_10min":"0.5"	"temperature_avg":"2.5"	"temperature_max":"3.8"	"temperature_min":"1.5"	"humidity_avg":"69"	"humidity_min":"38"	"wind_speed_avg":"3.1"	"wind_speed_max":"5"	"wind_direction":"北西"	"wind_speed_max_moment":"9.3"	"wind_direction_max_moment":"北西"	"sunshine_duration":"0"	"snowfall":""	"snowfall_max":""	"information_daytime":"時々雪後雨"	"information_night":"一時雨"


confファイルの設定では、mutate filterを使って入力をタブで分割したあとに、その内容をパースします。  
ファイル名は"ltsv.conf"とします。

```ruby:ltsv.conf
input{
  stdin{}
}
filter{
  mutate{
    split => {"message" => "    "}
  }
  grok{
    match => {
      "message" => "\"timestamp\":\"%{DATA:timestamp}\""
    }
  }
  grok{
    match => {
      "message" => "\"atmospheric_pressure\":\"%{DATA:atmospheric_pressure}\""
    }
  }
  grok{
    match => {
      "message" => "\"sea_level_pressure\":\"%{DATA:sea_level_pressure}\""
    }
  }
  grok{
    match => {
      "message" => "\"precipitation_day\":\"%{DATA:precipitation_day}\""
    }
  }
  grok{
    match => {
      "message" => "\"precipitation_max_hour\":\"%{DATA:precipitation_max_hour}\""
    }
  }
  grok{
    match => {
      "message" => "\"precipitation_max_10min\":\"%{DATA:precipitation_max_10min}\""
    }
  }
  grok{
    match => {
      "message" => "\"temperature_avg\":\"%{DATA:temperature_avg}\""
    }
  }
  grok{
    match => {
      "message" => "\"temperature_max\":\"%{DATA:temperature_max}\""
    }
  }
  grok{
    match => {
      "message" => "\"temperature_min\":\"%{DATA:temperature_min}\""
    }
  }
  grok{
    match => {
      "message" => "\"humidity_avg\":\"%{DATA:humidity_avg}\""
    }
  }
  grok{
    match => {
      "message" => "\"humidity_min\":\"%{DATA:humidity_min}\""
    }
  }
  grok{
    match => {
      "message" => "\"wind_speed_avg\":\"%{DATA:wind_speed_avg}\""
    }
  }
  grok{
    match => {
      "message" => "\"wind_speed_max\":\"%{DATA:wind_speed_max}\""
    }
  }
  grok{
    match => {
      "message" => "\"wind_direction\":\"%{DATA:wind_direction}\""
    }
  }
  grok{
    match => {
      "message" => "\"wind_direction_max_moment\":\"%{DATA:wind_direction_max_moment}\""
    }
  }
  grok{
    match => {
      "message" => "\"sunshine_duration\":\"%{DATA:sunshine_duration}\""
    }
  }
  grok{
    match => {
      "message" => "\"snowfall\":\"%{DATA:snowfall}\""
    }
  }
  grok{
    match => {
      "message" => "\"snowfall_max\":\"%{DATA:snowfall_max}\""
    }
  }
  grok{
    match => {
      "message" => "\"information_daytime\":\"%{DATA:information_daytime}\""
    }
  }
  grok{
    match => {
      "message" => "\"information_night\":\"%{DATA:information_night}\""
    }
  }
  date{
    match => ["timestamp","ISO8601"]
    timezone => "UTC"
    remove_field => ["timestamp","message"]
  }
}
output{
  elasticsearch{
    hosts => "XXX.XXX.XXX.232"
    index => "sample_index"
    document_type => "sample_ltsv"
  }
}
```

In [16]:
!ansible logstash-server -i ./hosts -m file -a "path=~/sample_data state=directory" -u $USER --private-key=$KEYPATH
!ansible logstash-server -i ./hosts -m file -a "path=~/sample_conf state=directory" -u $USER --private-key=$KEYPATH
!ansible logstash-server -i ./hosts -m copy -a "src=sample_data/weather.csv dest=~/sample_data/weather.ltsv" -u $USER --private-key=$KEYPATH
!ansible logstash-server -i ./hosts -m copy -a "src=logstash_conf/ltsv.conf dest=~/sample_conf/ltsv.conf" -u $USER --private-key=$KEYPATH

10.24.128.93 | SUCCESS => {
    "changed": false, 
    "gid": 1000, 
    "group": "centos", 
    "mode": "0775", 
    "owner": "centos", 
    "path": "/home/centos/sample_data", 
    "secontext": "unconfined_u:object_r:user_home_t:s0", 
    "size": 72, 
    "state": "directory", 
    "uid": 1000
}
10.24.128.93 | SUCCESS => {
    "changed": false, 
    "gid": 1000, 
    "group": "centos", 
    "mode": "0775", 
    "owner": "centos", 
    "path": "/home/centos/sample_conf", 
    "secontext": "unconfined_u:object_r:user_home_t:s0", 
    "size": 63, 
    "state": "directory", 
    "uid": 1000
}
10.24.128.93 | SUCCESS => {
    "changed": true, 
    "checksum": "c93efa572d2e1a05cf453c051587d8893247b5b7", 
    "dest": "/home/centos/sample_data/weather.ltsv", 
    "gid": 1000, 
    "group": "centos", 
    "md5sum": "dab98693b329f1917542f85d475c36af", 
    "mode": "0664", 
    "owner": "centos", 
    "secontext": "unconfined_u:object_r:user_home_t:s0", 
    "size": 2621, 
    "src": "/home/cent

実際のデータ登録は次のコマンドで実行します。

In [17]:
!ansible logstash-server -i ./hosts -m shell -a "cat ~/sample_data/weather.ltsv | sudo /usr/share/logstash/bin/logstash --path.settings /etc/logstash -f ~/sample_conf/ltsv.conf" -u $USER --private-key=$KEYPATH

10.24.128.93 | SUCCESS | rc=0 >>
Sending Logstash logs to /var/log/logstash which is now configured via log4j2.properties.



コマンドの実行に成功したら、次の検索APIで正しく登録されているか確認してください。  
正しく登録されていれば、json形式で登録されたデータが出力されます。

In [18]:
%%bash
curl -XGET "http://$ES_HOST/$INDEX/_search"

{"took":1,"timed_out":false,"_shards":{"total":1,"successful":1,"failed":0},"hits":{"total":121,"max_score":1.0,"hits":[{"_index":"sample_index","_type":"sample_json","_id":"AVyGu-TXdDy3gmLO23ZT","_score":1.0,"_source":{"precipitation_max_10min":"0","wind_speed_avg":"4.4","snowfall":"","precipitation_max_hour":"0","temperature_max":"8.2","wind_direction":"西","information_daytime":"時々晴一時雪","precipitation_day":"0","@timestamp":"2017-06-08T08:04:39.318Z","humidity_avg":"41","humidity_min":"30","wind_speed_max":"9.4","sunshine_duration":"1.3","temperature_avg":"3.7","temperature_min":"0.7","@version":"1","host":"0.0.0.0","atmospheric_pressure":"1000.4","wind_direction_max_moment":"北西","wind_speed_max_moment":"18.7","snowfall_max":"","sea_level_pressure":"1003.4","information_night":"晴","timestamp":"1"}},{"_index":"sample_index","_type":"sample_json","_id":"AVyGu-TXdDy3gmLO23ZU","_score":1.0,"_source":{"precipitation_max_10min":"","wind_speed_avg":"2.1","snowfall":"","precipitation_max_hour

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  7111  100  7111    0     0  1644k      0 --:--:-- --:--:-- --:--:-- 1736k


### 任意のテキスト(1行完結)

ここまでの説明のように、JSONやCSVなどの定型のデータであれば設定は簡単ですが、実際に読み込みたいデータはそのように形式がそろっていないことも多くあります。

次のサンプルデータのように、「名前=値」という形式がスペース区切りで1行に並んでいるようなデータを登録することを考えます。

In [19]:
!head sample_data/single_line.txt

timestamp=2015-01-01T00:00:00 location=tokyo atmospheric_pressure=1000.4 sea_level_pressure=1003.4 precipitation_day=0 precipitation_max_hour=0 precipitation_max_10min=0 temperature_avg=3.7 temperature_max=8.2 temperature_min=0.7 humidity_avg=41.0 humidity_min=30.0 wind_speed_avg=4.4 wind_speed_max=9.4 wind_direction=西 wind_speed_max_moment=18.7 wind_direction_max_moment=北西 sunshine_duration=1.3 information_daytime=時々晴一時雪 information_night=晴

この形式のデータを読み込むためのconfファイルは、次のような内容になります。  
ファイル名は"single_line.conf"とします。

```ruby:single_line.conf
input{
  stdin{}
}

filter{
  grok{
    match => {"message" => "timestamp=%{TIMESTAMP_ISO8601:timestamp}"}
  }
  grok{
    match => {"message" => "location=%{WORD:location}"}
  }
  grok{
    match => {"message" => "atmospheric_pressure=%{WORD:atmospheric_pressure}"}
  }
  grok{
    match => {"message" => "sea_level_pressure=%{WORD:sea_level_pressure}"}
  }
  grok{
    match => {"message" => "precipitation_day=%{WORD:precipitation_day}"}
  }
  grok{
    match => {"message" => "precipitation_max_hour=%{WORD:precipitation_max_hour}"}
  }
  grok{
    match => {"message" => "precipitation_max_10min=%{WORD:precipitation_max_10min}"}
  }
  grok{
    match => {"message" => "temperature_avg=%{WORD:temperature_avg}"}
  }
  grok{
    match => {"message" => "temperature_max=%{WORD:temperature_max}"}
  }
  grok{
    match => {"message" => "temperature_min=%{WORD:temperature_min}"}
  }
  grok{
    match => {"message" => "humidity_avg=%{WORD:humidity_avg}"}
  }
  grok{
    match => {"message" => "humidity_min=%{WORD:humidity_min}"}
  }
  grok{
    match => {"message" => "wind_speed_avg=%{WORD:wind_speed_avg}"}
  }
  grok{
    match => {"message" => "wind_speed_max=%{WORD:wind_speed_max}"}
  }
  grok{
    match => {"message" => "wind_direction=%{WORD:wind_direction}"}
  }
  grok{
    match => {"message" => "wind_speed_max_moment=%{WORD:wind_speed_max_moment}"}
  }
  grok{
    match => {"message" => "wind_direction_max_moment=%{WORD:wind_direction_max_moment}"}
  }
  grok{
    match => {"message" => "sunshine_duration=%{WORD:sunshine_duration}"}
  }
  grok{
    match => {"message" => "snowfall=%{WORD:snowfall}"}
  }
  grok{
    match => {"message" => "snowfall_max=%{WORD:snowfall_max}"}
  }
  grok{
    match => {"message" => "information_daytime=%{WORD:information_daytime}"}
  }
  grok{
    match => {"message" => "information_night=%{WORD:information_night}"}
  }
  date{
    match => ["timestamp","ISO8601"]
    timezone => "UTC"
    remove_field => "timestamp"
  }
}

output{
  elasticsearch{
    hosts => <ES_HOST>
    index => <INDEX>
  }
}
```

この設定では、2種類の加工処理を行っています。  
入力の特定部分をパースするための「grok filter」と  
文字列をタイムスタンプとして認識させるための「date filter」を用いています。  
加工処理の詳細に関しては、Logstashの[公式ページ](https://www.elastic.co/guide/en/logstash/current/filter-plugins.html)を参照してください。

実際のデータ登録は次のコマンドで実行します。

In [20]:
!ansible logstash-server -i ./hosts -m file -a "path=~/sample_data state=directory" -u $USER --private-key=$KEYPATH
!ansible logstash-server -i ./hosts -m file -a "path=~/sample_conf state=directory" -u $USER --private-key=$KEYPATH
!ansible logstash-server -i ./hosts -m copy -a "src=sample_data/single_line.txt dest=~/sample_data/single_line.txt" -u $USER --private-key=$KEYPATH
!ansible logstash-server -i ./hosts -m copy -a "src=logstash_conf/single_line.conf dest=~/sample_conf/single_line.conf" -u $USER --private-key=$KEYPATH

10.24.128.93 | SUCCESS => {
    "changed": false, 
    "gid": 1000, 
    "group": "centos", 
    "mode": "0775", 
    "owner": "centos", 
    "path": "/home/centos/sample_data", 
    "secontext": "unconfined_u:object_r:user_home_t:s0", 
    "size": 91, 
    "state": "directory", 
    "uid": 1000
}
10.24.128.93 | SUCCESS => {
    "changed": false, 
    "gid": 1000, 
    "group": "centos", 
    "mode": "0775", 
    "owner": "centos", 
    "path": "/home/centos/sample_conf", 
    "secontext": "unconfined_u:object_r:user_home_t:s0", 
    "size": 79, 
    "state": "directory", 
    "uid": 1000
}
10.24.128.93 | SUCCESS => {
    "changed": true, 
    "checksum": "fc112afd578cbfd46b09fc8edfe86bdb7b30cbb5", 
    "dest": "/home/centos/sample_data/single_line.txt", 
    "gid": 1000, 
    "group": "centos", 
    "md5sum": "924dc6d727a8767fd59b6203ddb83048", 
    "mode": "0664", 
    "owner": "centos", 
    "secontext": "unconfined_u:object_r:user_home_t:s0", 
    "size": 465, 
    "src": "/home/ce

In [21]:
!ansible logstash-server -i ./hosts -m shell -a "cat ~/sample_data/single_line.txt | sudo /usr/share/logstash/bin/logstash --path.settings /etc/logstash -f ~/sample_conf/single_line.conf" -u $USER --private-key=$KEYPATH

10.24.128.93 | SUCCESS | rc=0 >>
Sending Logstash logs to /var/log/logstash which is now configured via log4j2.properties.



コマンドの実行に成功したら、次の検索APIで正しく登録されているか確認してください。  
正しく登録されていれば、json形式で登録されたデータが出力されます。

In [22]:
%%bash
curl -XGET "http://$ES_HOST/$INDEX/sample_single_line/_search?pretty"

{
  "took" : 1,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "failed" : 0
  },
  "hits" : {
    "total" : 0,
    "max_score" : null,
    "hits" : [ ]
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   193  100   193    0     0  53865      0 --:--:-- --:--:-- --:--:-- 64333


### 任意のテキスト(複数行完結)


前の節の形式に加え、データが複数行にまたがっているような場合であっても、Logstashは対応が可能です。  
次のサンプルのようなデータを登録してみましょう。

In [23]:
!head -n 3 sample_data/multi_line.txt

timestamp=2015-01-01T00:00:00 location=tokyo atmospheric_pressure=1000.4
sea_level_pressure=1003.4 precipitation_day=0 precipitation_max_hour=0
precipitation_max_10min=0 temperature_avg=3.7 temperature_max=8.2


この形式のデータを読み込むためのconfファイルは、次のような内容になります。  
ファイル名は"multi_line.conf"とします。

```ruby:multi_line.conf
input{
  stdin{
    codec => multiline{
        pattern => "information_night"
        negate => "true"
        what => "next"
    }
  }
}

filter{
  grok{
    match => {"message" => "timestamp=%{TIMESTAMP_ISO8601:timestamp}"}
  }
  grok{
    match => {"message" => "location=%{WORD:location}"}
  }
  grok{
    match => {"message" => "atmospheric_pressure=%{WORD:atmospheric_pressure}"}
  }
  grok{
    match => {"message" => "sea_level_pressure=%{WORD:sea_level_pressure}"}
  }
  grok{
    match => {"message" => "precipitation_day=%{WORD:precipitation_day}"}
  }
  grok{
    match => {"message" => "precipitation_max_hour=%{WORD:precipitation_max_hour}"}
  }
  grok{
    match => {"message" => "precipitation_max_10min=%{WORD:precipitation_max_10min}"}
  }
  grok{
    match => {"message" => "temperature_avg=%{WORD:temperature_avg}"}
  }
  grok{
    match => {"message" => "temperature_max=%{WORD:temperature_max}"}
  }
  grok{
    match => {"message" => "temperature_min=%{WORD:temperature_min}"}
  }
  grok{
    match => {"message" => "humidity_avg=%{WORD:humidity_avg}"}
  }
  grok{
    match => {"message" => "humidity_min=%{WORD:humidity_min}"}
  }
  grok{
    match => {"message" => "wind_speed_avg=%{WORD:wind_speed_avg}"}
  }
  grok{
    match => {"message" => "wind_speed_max=%{WORD:wind_speed_max}"}
  }
  grok{
    match => {"message" => "wind_direction=%{WORD:wind_direction}"}
  }
  grok{
    match => {"message" => "wind_speed_max_moment=%{WORD:wind_speed_max_moment}"}
  }
  grok{
    match => {"message" => "wind_direction_max_moment=%{WORD:wind_direction_max_moment}"}
  }
  grok{
    match => {"message" => "sunshine_duration=%{WORD:sunshine_duration}"}
  }
  grok{
    match => {"message" => "snowfall=%{WORD:snowfall}"}
  }
  grok{
    match => {"message" => "snowfall_max=%{WORD:snowfall_max}"}
  }
  grok{
    match => {"message" => "information_daytime=%{WORD:information_daytime}"}
  }
  grok{
    match => {"message" => "information_night=%{WORD:information_night}"}
  }
  date{
    match => ["timestamp","ISO8601"]
    timezone => "UTC"
    remove_field => "timestamp"
  }
}

output{
  elasticsearch{
    hosts => "XXX.XXX.XXX.1:9200"
    index => "sample_index"
  }
}
```

1行で完結するデータを読み込むときとの差分はcodecでmultilineの設定をしているところです。  
Logstashでは通常、入力を１行単位で読み込んで処理します。  
multilineの指定をすることで、複数行を一つのイベントとして連結して処理することができます。

実際のデータ登録は次のコマンドで実行します。

In [24]:
!ansible logstash-server -i ./hosts -m file -a "path=~/sample_data state=directory" -u $USER --private-key=$KEYPATH
!ansible logstash-server -i ./hosts -m file -a "path=~/sample_conf state=directory" -u $USER --private-key=$KEYPATH
!ansible logstash-server -i ./hosts -m copy -a "src=sample_data/multi_line.txt dest=~/sample_data/multi_line.txt" -u $USER --private-key=$KEYPATH
!ansible logstash-server -i ./hosts -m copy -a "src=logstash_conf/multi_line.conf dest=~/sample_conf/multi_line.conf" -u $USER --private-key=$KEYPATH

10.24.128.93 | SUCCESS => {
    "changed": false, 
    "gid": 1000, 
    "group": "centos", 
    "mode": "0775", 
    "owner": "centos", 
    "path": "/home/centos/sample_data", 
    "secontext": "unconfined_u:object_r:user_home_t:s0", 
    "size": 4096, 
    "state": "directory", 
    "uid": 1000
}
10.24.128.93 | SUCCESS => {
    "changed": false, 
    "gid": 1000, 
    "group": "centos", 
    "mode": "0775", 
    "owner": "centos", 
    "path": "/home/centos/sample_conf", 
    "secontext": "unconfined_u:object_r:user_home_t:s0", 
    "size": 102, 
    "state": "directory", 
    "uid": 1000
}
10.24.128.93 | SUCCESS => {
    "changed": true, 
    "checksum": "6b40e1b0e0054c035cf0bde73c5b4ea92dd70c88", 
    "dest": "/home/centos/sample_data/multi_line.txt", 
    "gid": 1000, 
    "group": "centos", 
    "md5sum": "d90328884f60bc638ee617c1a17b89e0", 
    "mode": "0664", 
    "owner": "centos", 
    "secontext": "unconfined_u:object_r:user_home_t:s0", 
    "size": 838, 
    "src": "/home/

In [25]:
!ansible logstash-server -i ./hosts -m shell -a "cat ~/sample_data/multi_line.txt | sudo /usr/share/logstash/bin/logstash --path.settings /etc/logstash -f ~/sample_conf/multi_line.conf" -u $USER --private-key=$KEYPATH

10.24.128.93 | SUCCESS | rc=0 >>
Sending Logstash logs to /var/log/logstash which is now configured via log4j2.properties.



コマンドの実行に成功したら、次の検索APIで正しく登録されているか確認してください。  
正しく登録されていれば、json形式で登録されたデータが出力されます。

In [26]:
%%bash
curl -XGET "http://$ES_HOST/$INDEX/sample_multi_line/_search?pretty"

{
  "took" : 1,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "failed" : 0
  },
  "hits" : {
    "total" : 1,
    "max_score" : 1.0,
    "hits" : [
      {
        "_index" : "sample_index",
        "_type" : "sample_multi_line",
        "_id" : "AVyGvViQdDy3gmLO23bM",
        "_score" : 1.0,
        "_source" : {
          "precipitation_max_10min" : "0",
          "wind_speed_avg" : "4",
          "precipitation_max_hour" : "0",
          "temperature_max" : "8",
          "precipitation_day" : "0",
          "tags" : [
            "_grokparsefailure"
          ],
          "@timestamp" : "2015-01-01T00:00:00.000Z",
          "humidity_avg" : "41",
          "humidity_min" : "30",
          "wind_speed_max" : "9",
          "sunshine_duration" : "1",
          "temperature_avg" : "3",
          "temperature_min" : "0",
          "@version" : "1",
          "host" : "0.0.0.0",
          "atmospheric_pressure" : "1000",
          "location" : "tokyo"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1114  100  1114    0     0   233k      0 --:--:-- --:--:-- --:--:--  271k


## データ加工


ここまでの例では、読み込んだ値を特に加工せず、そのまま格納していました。  
この章では、confファイルの"filter"を定義することで値を加工する方法について説明します。

### geoipの付与

geoip filterを用いることで、ipアドレスをもとに地理情報を付与することができます。  
利用するサンプルデータは次のコマンドを実行することで確認できます。

In [27]:
!head -n 4 sample_data/geoip.json

{
  "recordtime":"2016-12-02",
  "clientip":"12.112.11.134"
}


具体的な設定を次に示します。

In [28]:
!cat logstash_conf/geoip.conf

input{
  stdin{
    codec => multiline{
      pattern => "}"
      negate => "true"
      what => "next"
    }
  }
}

filter{
  json{
    source => "message"
  }
  date{
    match => ["recordtime","yyyy-MM-dd"]
    remove_field => ["recordtime","message","host"]
  }
  geoip{
    source => "clientip"
  }
}

output{
  elasticsearch{
    hosts => "localhost:9200"
    index => "sample_index"
    document_type => "sample_geoip"
  }
}


In [29]:
!ansible logstash-server -i ./hosts -m file -a "path=~/sample_data state=directory" -u $USER --private-key=$KEYPATH
!ansible logstash-server -i ./hosts -m file -a "path=~/sample_conf state=directory" -u $USER --private-key=$KEYPATH
!ansible logstash-server -i ./hosts -m copy -a "src=sample_data/geoip.json dest=~/sample_data/geoip.json" -u $USER --private-key=$KEYPATH
!ansible logstash-server -i ./hosts -m copy -a "src=logstash_conf/geoip.conf dest=~/sample_conf/geoip.conf" -u $USER --private-key=$KEYPATH

10.24.128.93 | SUCCESS => {
    "changed": false, 
    "gid": 1000, 
    "group": "centos", 
    "mode": "0775", 
    "owner": "centos", 
    "path": "/home/centos/sample_data", 
    "secontext": "unconfined_u:object_r:user_home_t:s0", 
    "size": 4096, 
    "state": "directory", 
    "uid": 1000
}
10.24.128.93 | SUCCESS => {
    "changed": false, 
    "gid": 1000, 
    "group": "centos", 
    "mode": "0775", 
    "owner": "centos", 
    "path": "/home/centos/sample_conf", 
    "secontext": "unconfined_u:object_r:user_home_t:s0", 
    "size": 4096, 
    "state": "directory", 
    "uid": 1000
}
10.24.128.93 | SUCCESS => {
    "changed": true, 
    "checksum": "f70e48628f5900284f71c010bb2208f2ee57d373", 
    "dest": "/home/centos/sample_data/geoip.json", 
    "gid": 1000, 
    "group": "centos", 
    "md5sum": "e449125f2b32622791447d44552b0446", 
    "mode": "0664", 
    "owner": "centos", 
    "secontext": "unconfined_u:object_r:user_home_t:s0", 
    "size": 794, 
    "src": "/home/cen

geoip{}の中のsourceという項目に、地理情報を取得したいipアドレスを指定します。  

実際のデータ登録は次のセルのコマンドを実行してください。  

In [30]:
!ansible logstash-server -i ./hosts -m shell -a "cat ~/sample_data/geoip.json | sudo /usr/share/logstash/bin/logstash --path.settings /etc/logstash -f ~/sample_conf/geoip.conf" -u $USER --private-key=$KEYPATH

10.24.128.93 | SUCCESS | rc=0 >>
Sending Logstash logs to /var/log/logstash which is now configured via log4j2.properties.



コマンドの実行に成功したら、次の検索APIで正しく登録されているか確認してください。  
もとのデータに含まれている**recordtime**,**clientip**以外に国名や緯度・経度などが追加されています。

In [31]:
%%bash
curl -XGET "http://$ES_HOST/$INDEX/_search?pretty" -d @- <<EOF
{
    "size":"1",
        "query":{
            "match":{"_type":"sample_geoip"}
            }
}
EOF

{
  "took" : 3,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "failed" : 0
  },
  "hits" : {
    "total" : 13,
    "max_score" : 1.0,
    "hits" : [
      {
        "_index" : "sample_index",
        "_type" : "sample_geoip",
        "_id" : "AVyGvaAvdDy3gmLO23bN",
        "_score" : 1.0,
        "_source" : {
          "@timestamp" : "2016-12-02T00:00:00.000Z",
          "geoip" : {
            "timezone" : null,
            "ip" : "12.112.11.134",
            "latitude" : 37.751,
            "continent_code" : "NA",
            "city_name" : null,
            "country_code2" : "US",
            "country_name" : "United States",
            "dma_code" : null,
            "country_code3" : "US",
            "region_name" : null,
            "location" : [
              -97.822,
              37.751
            ],
            "postal_code" : null,
            "longitude" : -97.822,
            "region_code" : null
          },
          "clientip" : "

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1222  100  1131  100    91   160k  13253 --:--:-- --:--:-- --:--:--  184k


### useragentの解析

useragentの情報を解析するためには、useragent filterを利用します。  
利用するサンプルデータは次のコマンドを実行することで確認できます。

In [32]:
!head -n 3 sample_data/useragent.txt

Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8) AppleWebKit/536.25 (KHTML, like Gecko) Version/6.0 Safari/536.25
Mozilla/4.0 (compatible; MSIE 6.0; Nitro) Opera 8.50 [ja]
Mozilla/5.0 (Macintosh; Intel Mac OS X 10.7; rv:9.0.1) Gecko/20100101 Firefox/9.0.1


具体的な設定を次に示します。

In [33]:
!cat logstash_conf/useragent.conf

input{
  stdin{}
}
filter{
  useragent{
    source => "message"
    remove_field => ["message","host"]
  }
}
output{
  elasticsearch{
    hosts => "localhost:9200"
    index => "sample_index"
    document_type => "sample_useragent"
  }
}


In [34]:
!ansible logstash-server -i ./hosts -m file -a "path=~/sample_data state=directory" -u $USER --private-key=$KEYPATH
!ansible logstash-server -i ./hosts -m file -a "path=~/sample_conf state=directory" -u $USER --private-key=$KEYPATH
!ansible logstash-server -i ./hosts -m copy -a "src=sample_data/useragent.txt dest=~/sample_data/useragent.txt" -u $USER --private-key=$KEYPATH
!ansible logstash-server -i ./hosts -m copy -a "src=logstash_conf/useragent.conf dest=~/sample_conf/useragent.conf" -u $USER --private-key=$KEYPATH

10.24.128.93 | SUCCESS => {
    "changed": false, 
    "gid": 1000, 
    "group": "centos", 
    "mode": "0775", 
    "owner": "centos", 
    "path": "/home/centos/sample_data", 
    "secontext": "unconfined_u:object_r:user_home_t:s0", 
    "size": 4096, 
    "state": "directory", 
    "uid": 1000
}
10.24.128.93 | SUCCESS => {
    "changed": false, 
    "gid": 1000, 
    "group": "centos", 
    "mode": "0775", 
    "owner": "centos", 
    "path": "/home/centos/sample_conf", 
    "secontext": "unconfined_u:object_r:user_home_t:s0", 
    "size": 4096, 
    "state": "directory", 
    "uid": 1000
}
10.24.128.93 | SUCCESS => {
    "changed": true, 
    "checksum": "905770f57017cf2d8bd0e9346f8b97eb422191d5", 
    "dest": "/home/centos/sample_data/useragent.txt", 
    "gid": 1000, 
    "group": "centos", 
    "md5sum": "93eb6ecc79cdae56ea3f41f38b1032c6", 
    "mode": "0664", 
    "owner": "centos", 
    "secontext": "unconfined_u:object_r:user_home_t:s0", 
    "size": 432, 
    "src": "/home/

useragent filterを利用すると、自動的にuseragentの情報をパースすることができます。  

実際のデータ登録は次のセルのコマンドを実行してください。  

In [35]:
!ansible logstash-server -i ./hosts -m shell -a "cat ~/sample_data/useragent.txt | sudo /usr/share/logstash/bin/logstash --path.settings /etc/logstash -f ~/sample_conf/useragent.conf" -u $USER --private-key=$KEYPATH

10.24.128.93 | SUCCESS | rc=0 >>
Sending Logstash logs to /var/log/logstash which is now configured via log4j2.properties.



コマンドの実行に成功したら、次の検索APIで正しく登録されているか確認してください。  
もとのデータに含まれているrecordtime,clientip以外に国名や緯度・経度などが追加されています。

In [36]:
%%bash
curl -XGET "http://$ES_HOST/$INDEX/_search?pretty" -d @- <<EOF
{
    "size":"1",
        "query":{
            "match":{"_type":"sample_useragent"}
            }
}

{
  "took" : 1,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "failed" : 0
  },
  "hits" : {
    "total" : 4,
    "max_score" : 1.0,
    "hits" : [
      {
        "_index" : "sample_index",
        "_type" : "sample_useragent",
        "_id" : "AVyGveg7dDy3gmLO23ba",
        "_score" : 1.0,
        "_source" : {
          "@timestamp" : "2017-06-08T08:06:51.054Z",
          "os" : "Mac OS X 10.8",
          "major" : "6",
          "minor" : "0",
          "os_minor" : "8",
          "os_major" : "10",
          "@version" : "1",
          "name" : "Safari",
          "os_name" : "Mac OS X",
          "device" : "Other"
        }
      }
    ]
  }
}


bash: line 7: warning: here-document at line 1 delimited by end-of-file (wanted `EOF')
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   790  100   695  100    95   200k  28056 --:--:-- --:--:-- --:--:--  226k


## Ingest Nodeの利用方法

Ingest Nodeを利用するためには、まずpipelineを作成する必要があります。  

![images/03_pipeline.png](images/03_pipeline.png)


Node種類の説明で記述したように、pipelineはIngest Nodeが受け取ったドキュメントに対して、いくつかの加工処理(processor)を順次実行するものとなります。  

pipelineの作成には、Put Pipeline APIを用います。pipelineには次の二つの要素を定義します。  
* description : 処理内容の説明
* processors : 実際の処理

ここでは受け取ったドキュメントに対して新しいフィールドを追加するためのpipelineを作成します。  
ここで扱う例では、"country"というフィールドを追加し、値を"japan"に設定しています。

In [37]:
%%bash
curl -XPUT "http://$ES_HOST/_ingest/pipeline/my-pipeline" -d @- << EOF
{
  "description":"describe pipeline",
  "processors":[
    {
      "set":{
        "field":"country",
        "value": "japan"
      }
    }
  ]
}
EOF

{"acknowledged":true}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   158  100    21  100   137    672   4387 --:--:-- --:--:-- --:--:--  4419


作成済みpipelineの確認にはGet Ppeline APIを用います。次のコマンドで作成済みpipelineの確認が可能です。

In [38]:
%%bash
curl -XGET "http://$ES_HOST/_ingest/pipeline?pretty"

{
  "my-pipeline" : {
    "description" : "describe pipeline",
    "processors" : [
      {
        "set" : {
          "field" : "country",
          "value" : "japan"
        }
      }
    ]
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   199  100   199    0     0  93471      0 --:--:-- --:--:-- --:--:-- 99500


pipelineの動作をシミュレートするためには、Simulate Pipeline APIを用います。  
pipelineの定義と、ドキュメントの内容を記述することで、実際にpipeline処理が行われた時の出力結果を確認することができます。

In [39]:
%%bash
curl -XPOST "http://$ES_HOST/_ingest/pipeline/_simulate?pretty" -d @- << EOF
{
    "pipeline":{
        "description":"_description",
        "processors":[
            {
                "set":{
                    "field":"country",
                    "value":"japan"
                }
            }
        ]
    },
    "docs":[
        {
            "_index":"$INDEX",
            "_type":"$TYPE",
            "_id":"id",
            "_source":{
                "timestamp":"1",
                "atmospheric_pressure":"1000.4",
                "sea_level_pressure":"1003.4",
                "precipitation_day":"0",
                "precipitation_max_hour":"0",
                "precipitation_max_10min":"0",
                "temperature_avg":"3.7",
                "temperature_max":"8.2",
                "temperature_min":"0.7",
                "humidity_avg":"41",
                "humidity_min":"30",
                "wind_speed_avg":"4.4",
                "wind_speed_max":"9.4",
                "wind_direction":"西",
                "wind_speed_max_moment":"18.7",
                "wind_direction_max_moment":"北西",
                "sunshine_duration":"1.3",
                "snowfall":"",
                "snowfall_max":"",
                "information_daytime":"時々晴一時雪",
                "information_night":"晴"
            }
        },
        {
            "_index":"$INDEX",
            "_type":"$TYPE",
            "_id":"id",
            "_source":{
                "timestamp":"2",
                "atmospheric_pressure":"1007.4",
                "sea_level_pressure":"1010.4",
                "precipitation_day":"",
                "precipitation_max_hour":"",
                "precipitation_max_10min":"",
                "temperature_avg":"2.7",
                "temperature_max":"7.9",
                "temperature_min":"-2.2",
                "humidity_avg":"41",
                "humidity_min":"19",
                "wind_speed_avg":"2.1",
                "wind_speed_max":"4.2",
                "wind_direction":"西",
                "wind_speed_max_moment":"7.8",
                "wind_direction_max_moment":"西",
                "sunshine_duration":"6",
                "snowfall":"",
                "snowfall_max":"",
                "information_daytime":"時々曇",
                "information_night":"一時曇"
            }
        }
    ]
}
EOF

{
  "docs" : [
    {
      "doc" : {
        "_type" : "logs",
        "_id" : "id",
        "_index" : "sample_index",
        "_source" : {
          "country" : "japan",
          "wind_speed_avg" : "4.4",
          "snowfall" : "",
          "temperature_max" : "8.2",
          "precipitation_day" : "0",
          "wind_speed_max" : "9.4",
          "sunshine_duration" : "1.3",
          "temperature_avg" : "3.7",
          "temperature_min" : "0.7",
          "wind_direction_max_moment" : "北西",
          "information_night" : "晴",
          "timestamp" : "1",
          "precipitation_max_10min" : "0",
          "precipitation_max_hour" : "0",
          "wind_direction" : "西",
          "information_daytime" : "時々晴一時雪",
          "humidity_avg" : "41",
          "humidity_min" : "30",
          "atmospheric_pressure" : "1000.4",
          "wind_speed_max_moment" : "18.7",
          "snowfall_max" : "",
          "sea_level_pressure" : "1003.4"
        },
        "_ingest" : {
     

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4431  100  2169  100  2262   300k   313k --:--:-- --:--:-- --:--:--  368k


## トラブルシューティング

### 文字コード指定

Logstashでデータを扱う際には、文字コードを指定することができます。  
codecプラグインの中でcharsetという項目を設定することで文字コードが指定されます。  
デフォルト設定はutf-8になっています。

文字コードをshift-jisに指定する場合の設定例を示します。

### 自動で入るフィールドの削除

Elasticsearchにドキュメントを登録する際に、自動で付加されるフィールドがいくつかあります。  
例えば\_allフィールドや\_sourceフィールドなどが自動的に付加されます。  
これらのフィールドは高速に検索を行う時などに有用なフィールドですが、削除することでindexのサイズを少なくすることもできます。  
_allや_sourceの詳細に関してはElastic Referenceの[Meta-Fields](https://www.elastic.co/guide/en/elasticsearch/reference/current/mapping-fields.html)を参照してください。

上記の_allや_sourceは、Elasticsearchへの登録時に作成されるフィールドですが、  
hostや@timestampなどのようにLogstashで自動的に作成されるフィールドもあります。  
これらのフィールドはLogstashの設定ファイルに設定を記述することで削除することができます。  
詳しくはElastic Referenceの[mutate](https://www.elastic.co/guide/en/logstash/current/plugins-filters-mutate.html#plugins-filters-mutate-remove_field)を参照してください。

### ドキュメントIDの重要性と付与方法

#### ドキュメントID の重要性

Elasticsearchに登録されるドキュメントは、ドキュメントごとに固有のドキュメントIDを持ちます。  
Logstashを用いてElasticsearchへのドキュメント登録を行う場合、ドキュメントIDを明示的に指定しなければランダムなドキュメントIDが付与されるようになっています。  
  
既にElasticsearchに登録されているものと重複するドキュメントIDを持ったドキュメントを新たに登録(index)しようとした場合には、ドキュメントは更新されません。  
新しいドキュメントで上書きする場合は、更新(update)する必要があります。  
この振る舞いは、index時の設定によって変更可能です。  
設定の変更方法については、Logstash ReferenceのOutputプラグインelasticsearchの[actionパラメータの説明](https://www.elastic.co/guide/en/logstash/current/plugins-outputs-elasticsearch.html#plugins-outputs-elasticsearch-action)を参照してください。
  
ドキュメントIDを明示的に指定することの利点として挙げられるのが、ドキュメントの再登録時の重複防止です。  
明示的に指定しなかった場合、内容が同一のドキュメントを複数回Elasticsearchに登録すると、その分だけ重複して登録されてしまいます。  
ドキュメントIDを指定しておくことで、ドキュメント登録の処理が中断してしまった場合でも重複を気にせず再登録することができます。

#### ドキュメントIDの付与方法

LogstashでドキュメントIDを明示的に付与するにはLogstashのoutputにある、elasticsearchプラグインの設定を利用します。  
ドキュメントIDという項目で直接指定することができます。  
他のフィールドの値を用いることもできるので、例えばmessageの内容をもとにドキュメントIDを付与すれば、同一内容のドキュメントに関して常に一意なidを付与することができます。  
具体的な設定は次の通りです。

In [40]:
!cat logstash_conf/document_id.conf

input{
  stdin{
  }
}

filter{
  csv{
    columns => ["timestamp","location","atmospheric_pressure","sea_level_pressure","precipitation_day","precipitation_max_hour","precipitation_max_10min","temperature_avg","temperature_max","temperature_min","humidity_avg","humidity_min","wind_speed_avg","wind_speed_max","wind_direction","wind_speed_max_moment","wind_direction_max_moment","sunshine_duration","snowfall","snowfall_max","information_daytime","information_night"]
  }
  date{
    match => ["timestamp","ISO8601"]
    remove_field => "timestamp"
  }
}

output{
  elasticsearch{
    index => "document_id_test"
    document_id => "%{location}-%{@timestamp}"
    hosts => "localhost"
  }
}

### 処理時間算出

データ投入作業時にかかる時間を、概算で算出する方法を示します。  
やり方として、定期的にElasticsearchへクエリを送り、単位時間あたりどのくらいデータ投入ができているかを計測し算出します。  
次の全件検索コマンドを実行します。

In [41]:
%%bash
curl -XGET "http://$ES_HOST/$INDEX/_search?pretty" -d @- << EOF
{
  "size": 0,
  "query" : { 
    "match_all" : {} 
  } 
}
EOF

{
  "took" : 1,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "failed" : 0
  },
  "hits" : {
    "total" : 139,
    "max_score" : 0.0,
    "hits" : [ ]
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   247  100   194  100    53  53296  14560 --:--:-- --:--:-- --:--:-- 64666


"hits"配下の"total"が投入されたデータ数になり、この値を使用します。  
例えば、ある程度の間隔でコマンドを実行し、totalの変化量を知ることでおおまかにですが全体の処理時間を算出することができます。  

### タイムゾーンについての注意

Elasticsearchの内部ではデータの時間をUTCで持っています。  
例えば、インダックス名に"some_index-2015.01.01"のような日付を使用していた場合、日本時間で1月1日8時のデータは"some_index-2014.12.31"の23時のデータとして持ちます。9時間ずれた結果、前日のインデックスに登録されます。インデックスを指定して検索する場合には注意が必要です。  
  
データ投入時に使用したいタイムゾーンに合わせる必要があります。  
データ投入後にタイムゾーンを変更する場合、再投入が必要となります。

タイムゾーンを指定したクエリについてはElasticsearch Referenceの[Time Zone](https://www.elastic.co/guide/en/elasticsearch/reference/5.0/search-aggregations-bucket-datehistogram-aggregation.html#_time_zone)を参照してください。